In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
import cleaned_econ_data

In [ ]:
STEPS = 5000
PRICE_NORM_FACTOR = 1000

In [ ]:
tf.logging.set_verbosity(tf.logging.INFO)

"""Builds, trains, and evaluates the model."""
(train, test) = cleaned_econ_data.dataset()

# Switch the labels to units of thousands for better convergence.
def normalize_price(features, labels):
    return features, labels / PRICE_NORM_FACTOR

train = train.map(normalize_price)
test = test.map(normalize_price)

# Build the training input_fn.
def input_train():
    return (
            # Shuffling with a buffer larger than the data set ensures
            # that the examples are well mixed.
            train.shuffle(1000).batch(128)
            # Repeat forever
            .repeat().make_one_shot_iterator().get_next())

# Build the validation input_fn.
def input_test():
    return (test.shuffle(1000).batch(128)
                    .make_one_shot_iterator().get_next())

feature_columns = [
        tf.feature_column.numeric_column(key="funds-rate"),
        tf.feature_column.numeric_column(key="oil-cpi"),
        tf.feature_column.numeric_column(key="high-tax-rate"),
        tf.feature_column.numeric_column(key="low-tax-rate"),
        tf.feature_column.numeric_column(key="unemployment"),
        tf.feature_column.numeric_column(key="snp")
]

# Build a DNNRegressor, with 2x20-unit hidden layers, with the feature columns
# defined above as input.
model = tf.estimator.DNNRegressor(
        hidden_units=[20, 20], feature_columns=feature_columns)

# Train the model.
model.train(input_fn=input_train, steps=STEPS)

# Evaluate how the model performs on data it has not yet seen.
eval_result = model.evaluate(input_fn=input_test)

# The evaluation returns a Python dictionary. The "average_loss" key holds the
# Mean Squared Error (MSE).
average_loss = eval_result["average_loss"]

# Convert MSE to Root Mean Square Error (RMSE).
print("\n" + 80 * "*")
print("\nRMS error for the test set: {:.0f}".format(average_loss**0.5))
print(eval_result)

print()